# Predicción de crímenes
### Trabajo Fin de Máster para el Máster Universitario en Ciencia de Datos
### Universitat Oberta de Catalunya
### Realizado por Álvaro Pavón Díaz

## Parte 2: Modelado y evaluación de los datos
Durante este Jupyter Notebook nos dedicaremos a realizar el entrenamiento de los conjunto de datos sobre diferentes métodos.

Para ello lo primero se va a proceder a importar las diferentes clases que se necesitarán a lo largo de este notebook y la importación de los archivos que se va a utilizar. Además de mostrar que los proceso realizados a través de TensorFlow van a ser acelerados mediante GPU (GeForce RTX 2070 SUPER)

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time_function
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

print(device_lib.list_local_devices())
print(tf.version)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9304242234893452707
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10687349824867975108
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 469631406589028743
physical_device_desc: "device: XLA_GPU device"
]
<module 'tensorflow._api.v2.version' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [2]:
crimes = pd.read_csv ('crimes_clean.csv', sep='""', delimiter=',', engine='python')
crimes.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3
1,Property Lost,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30
2,Other,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3
4,Aircraft,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3


In [3]:
crimes_time = pd.read_csv ('crimes_clean_time.csv', sep='""', delimiter=',', engine='python')
crimes_time.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH,TIME_X,TIME_Y,Lat_M,Long_M
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3,0.518198,0.000331,0.185665,0.267386
1,Property Lost,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30,0.239106,0.073462,0.736023,0.201687
2,Other,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3,0.517840,0.000318,0.465558,0.473233
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3,0.518198,0.000331,0.781170,0.553614
4,Aircraft,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3,0.518198,0.000331,0.878350,0.716335


In [4]:
crimes_time=crimes_time[['OFFENSE_CODE_GROUP', 'SHOOTING','UCR_PART','YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','HOUR','IS_NIGHT','Lat', 'Lat_M', 'Long','Long_M', 'TIME_X', 'TIME_Y', 'DISTRICT']]
crimes_time.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,UCR_PART,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
0,Disorderly Conduct,0.0,Part Two,2018,10,3,Wednesday,20,1,42.262608,0.185665,-71.121186,0.267386,0.518198,0.000331,E18
1,Property Lost,0.0,Part Three,2018,8,30,Thursday,20,1,42.352111,0.736023,-71.135311,0.201687,0.239106,0.073462,D14
2,Other,0.0,Part Two,2018,10,3,Wednesday,19,1,42.308126,0.465558,-71.076930,0.473233,0.517840,0.000318,B2
3,Aggravated Assault,0.0,Part One,2018,10,3,Wednesday,20,1,42.359454,0.781170,-71.059648,0.553614,0.518198,0.000331,A1
4,Aircraft,0.0,Part Three,2018,10,3,Wednesday,20,1,42.375258,0.878350,-71.024663,0.716335,0.518198,0.000331,A7


In [5]:
crimes_times = crimes_time.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_times.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,UCR_PART,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
127095,Harassment,0.0,Part Two,2015,6,15,Monday,0,1,42.291093,0.360820,-71.065945,0.524324,0.022469,0.648202,C11
241630,Property Lost,0.0,Part Three,2015,6,15,Monday,0,1,42.283634,0.314958,-71.082813,0.445868,0.022469,0.648202,B3
288821,Fraud,0.0,Part Two,2015,6,15,Monday,0,1,42.360205,0.785788,-71.056208,0.569614,0.022469,0.648202,A1
295842,Other,0.0,Part Two,2015,6,15,Monday,0,1,42.293606,0.376273,-71.071887,0.496690,0.022469,0.648202,C11
300018,Confidence Games,0.0,Part Two,2015,6,15,Monday,0,1,42.300217,0.416923,-71.080979,0.454397,0.022469,0.648202,B3


In [6]:
crimes_time_one = crimes_time
le_dw = LabelEncoder()
dw_labels = le_dw.fit_transform(crimes_time_one['DAY_OF_WEEK'])
crimes_time_one['DAY_OF_WEEK_label'] = dw_labels

one_dw = OneHotEncoder()
days_of_week = one_dw.fit_transform(crimes_time_one[['DAY_OF_WEEK_label']]).toarray()
days_of_week_labels = list(le_dw.classes_)
days_of_week_df = pd.DataFrame(days_of_week, columns=days_of_week_labels)



In [7]:
le_ocg = LabelEncoder()
ocg_labels = le_ocg.fit_transform(crimes_time_one['OFFENSE_CODE_GROUP'])
crimes_time_one['OFFENSE_CODE_label'] = ocg_labels

one_ocg = OneHotEncoder()
ocg = one_ocg.fit_transform(crimes_time_one[['OFFENSE_CODE_label']]).toarray()
ocg_labels = list(le_ocg.classes_)
ocg_df = pd.DataFrame(ocg, columns=ocg_labels)

In [8]:
#le_year = LabelEncoder()
#year_labels = le_year.fit_transform(crimes_time_one['YEAR'])
#crimes_time_one['year_label'] = year_labels

#one_year = OneHotEncoder()
#year = one_year.fit_transform(crimes_time_one[['year_label']]).toarray()
#year_labels = list(le_year.classes_)
#year_df = pd.DataFrame(year, columns=year_labels)

In [9]:
le_d = LabelEncoder()
d_labels = le_d.fit_transform(crimes_time_one['DISTRICT'])
crimes_time_one['DISTRICT_labels'] = d_labels

In [10]:
le_ucr = LabelEncoder()
ucr_labels = le_ucr.fit_transform(crimes_time_one['UCR_PART'])
crimes_time_one['UCR_PART_label'] = ucr_labels

one_ucr = OneHotEncoder()
ucr = one_ucr.fit_transform(crimes_time_one[['UCR_PART_label']]).toarray()
ucr_labels = list(le_ucr.classes_)
ucr_df = pd.DataFrame(ucr, columns=ucr_labels)

In [11]:
crimes_time_one = pd.concat([ocg_df,ucr_df,days_of_week_df, crimes_time_one], axis=1)

crimes_time_one.drop('DAY_OF_WEEK_label', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_label', 1, inplace= True)
crimes_time_one.drop('DAY_OF_WEEK', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_GROUP', 1, inplace= True)
#crimes_time_one.drop('year_label', 1, inplace= True)
crimes_time_one.drop('DISTRICT', 1, inplace= True)
crimes_time_one.drop('Lat', 1, inplace= True)
crimes_time_one.drop('Long', 1, inplace= True)
crimes_time_one.drop('Lat_M', 1, inplace= True)
crimes_time_one.drop('Long_M', 1, inplace= True)
crimes_time_one.drop('UCR_PART', 1, inplace= True)

In [12]:
crimes_time_one = crimes_time_one.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_time_train_test = crimes_time_one.head(int(crimes_time_one.shape[0] * 0.8))
crimes_time_validation = crimes_time_one.tail(crimes_time_one.shape[0] - int(crimes_times.shape[0] * 0.8))
crimes_time_train_test.drop('YEAR', 1, inplace= True)
crimes_time_validation.drop('YEAR', 1, inplace= True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
crimes_time_train_test.head(5)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels,UCR_PART_label
127095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,5,2
241630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,4,1
288821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,0,2
295842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,5,2
300018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,4,2


In [14]:
crimes_time_validation.head(5)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels,UCR_PART_label
61407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,4,1
61408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,7,0
61411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2
61412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2
61413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2


In [15]:
crimes_time_validation.shape

(60846, 78)

In [16]:
crimes_time_train_test.shape

(243383, 78)

In [17]:
xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'Aggravated Assault':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT_labels'], test_size = 0.25, random_state = 0)
#xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'OFFENSE_CODE_GROUP':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT'], test_size = 0.25, random_state = 0)

In [18]:
#xTrain.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
#xTest.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
xTrain.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
xTest.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)

In [19]:
xTrain.head(15)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,SHOOTING,IS_NIGHT,TIME_X,TIME_Y
231328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.489538,0.999891
302995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.012670,0.611847
195984,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.157651,0.135587
88225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.605984,0.011362
289842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.108869,0.188526
82894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.232153,0.077794
212207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.017742,0.632013
102925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.168547,0.125649
196485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.121758,0.172994
290356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.097312,0.203618


In [20]:
len(list(le_ocg.classes_))

59

In [21]:
def create_model (type_layer= 'LSTM', n_layers=1, n_embeding = 32,n_epochs = 20,batch_size=512, top_k=3):
    
    layer_r = tf.keras.layers.LSTM(n_embeding, return_sequences=True)
    layer_e = tf.keras.layers.LSTM(n_embeding)
    if type_layer == 'GRU':
        layer_r = tf.keras.layers.GRU(n_embeding, return_sequences=True)
        layer_e = tf.keras.layers.GRU(n_embeding)
        
    model_i = tf.keras.models.Sequential()
    model_i.add(tf.keras.layers.Embedding(30, n_embeding, input_length=73))
    if n_layers != 0:
        for i in range(n_layers):
            model_i.add(layer_r)
    
    model_i.add(layer_e)
    
    model_i.add(tf.keras.layers.Dense(12))
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    metric= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
    model_i.compile (optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics=[metric] )
    
    return model_i.fit(xTrain, yTrain, epochs = n_epochs,batch_size=batch_size, validation_data=(xTest, yTest))


In [22]:
create_model (type_layer= 'LSTM', n_layers=0, n_embeding = 128,n_epochs = 20,batch_size=256 )

Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 11s 59us/sample - loss: 3.8835 - sparse_top_k_categorical_accuracy: 0.2896 - val_loss: 3.7897 - val_sparse_top_k_categorical_accuracy: 0.2943
Epoch 2/20
182537/182537 [==============================] - 9s 48us/sample - loss: 3.8055 - sparse_top_k_categorical_accuracy: 0.2924 - val_loss: 3.7897 - val_sparse_top_k_categorical_accuracy: 0.2943
Epoch 3/20
182537/182537 [==============================] - 9s 48us/sample - loss: 3.8055 - sparse_top_k_categorical_accuracy: 0.2924 - val_loss: 3.7897 - val_sparse_top_k_categorical_accuracy: 0.2943
Epoch 4/20
182537/182537 [==============================] - 9s 48us/sample - loss: 3.8055 - sparse_top_k_categorical_accuracy: 0.2924 - val_loss: 3.7897 - val_sparse_top_k_categorical_accuracy: 0.2943
Epoch 5/20
182537/182537 [==============================] - 9s 48us/sample - loss: 3.8055 - sparse_top_k_categorical_accuracy: 0.2924 - val_loss

In [22]:
def grid_search(n_layers=[0,1,2,3], type_layers= ['GRU','LSTM'], n_embedings=[64,128], n_epochs = 20, top_ks=[3,4]):
    i=0
    lista = {}
    for n_layer in n_layers:
        for type_l in type_layers:
            for n_embeding in n_embedings:
                for top_k in top_ks:
                    print("************ NUEVA PRUEBA *************")
                    print (type_l)
                    print (n_layer)
                    print (n_embeding)
                    print (top_k)
                    try:
                        fit = create_model (type_layer= type_l, n_layers=n_layer, n_embeding = n_embeding, n_epochs=n_epochs)
                        print(fit)
                        print(i)
                        lista[str(i)] = "type_l=" + type_l + ",n_layer="+ str(n_layer) + ",n_embeding" + str(n_embeding) + ",top_k=" + str(top_k)
                        i=i+1
                    except:
                        print("Imposible entrenar el modelos con los parametros siguientes")
    return lista



In [23]:
resultados0 = grid_search (n_layers=[0],type_layers= ['GRU','LSTM'], n_embedings=[64,128], top_ks=[3,4])


************ NUEVA PRUEBA *************
GRU
0
64
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 6s 33us/sample - loss: 4.5315 - sparse_top_k_categorical_accuracy: 0.3271 - val_loss: 4.4083 - val_sparse_top_k_categorical_accuracy: 0.3977
Epoch 2/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3845 - sparse_top_k_categorical_accuracy: 0.3895 - val_loss: 2.3604 - val_sparse_top_k_categorical_accuracy: 0.4065
Epoch 3/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3580 - sparse_top_k_categorical_accuracy: 0.4186 - val_loss: 2.3553 - val_sparse_top_k_categorical_accuracy: 0.4260
Epoch 4/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3555 - sparse_top_k_categorical_accuracy: 0.4250 - val_loss: 2.3531 - val_sparse_top_k_categorical_accuracy: 0.4263
Epoch 5/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3550 - sp

Epoch 19/20
182537/182537 [==============================] - 6s 34us/sample - loss: 2.4849 - sparse_top_k_categorical_accuracy: 0.1893 - val_loss: 2.4849 - val_sparse_top_k_categorical_accuracy: 0.1890
Epoch 20/20
182537/182537 [==============================] - 6s 34us/sample - loss: 2.4849 - sparse_top_k_categorical_accuracy: 0.1893 - val_loss: 2.4849 - val_sparse_top_k_categorical_accuracy: 0.1890
3
************ NUEVA PRUEBA *************
LSTM
0
64
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 5s 27us/sample - loss: 3.5039 - sparse_top_k_categorical_accuracy: 0.3258 - val_loss: 3.4637 - val_sparse_top_k_categorical_accuracy: 0.3239
Epoch 2/20
182537/182537 [==============================] - 4s 22us/sample - loss: 3.4515 - sparse_top_k_categorical_accuracy: 0.3271 - val_loss: 3.4640 - val_sparse_top_k_categorical_accuracy: 0.3239
Epoch 3/20
182537/182537 [==============================] - 4s 23us/sample - loss: 3.4510

Epoch 18/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3487 - sparse_top_k_categorical_accuracy: 0.4304 - val_loss: 2.3453 - val_sparse_top_k_categorical_accuracy: 0.4325
Epoch 19/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3554 - sparse_top_k_categorical_accuracy: 0.4134 - val_loss: 2.3577 - val_sparse_top_k_categorical_accuracy: 0.4043
Epoch 20/20
182537/182537 [==============================] - 4s 22us/sample - loss: 2.3590 - sparse_top_k_categorical_accuracy: 0.4020 - val_loss: 2.3592 - val_sparse_top_k_categorical_accuracy: 0.4043
5
************ NUEVA PRUEBA *************
LSTM
0
128
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 8s 43us/sample - loss: 3.1130 - sparse_top_k_categorical_accuracy: 0.3179 - val_loss: 2.4264 - val_sparse_top_k_categorical_accuracy: 0.3609
Epoch 2/20
182537/182537 [==============================] - 7s 37us/sample - loss: 2.42

Epoch 17/20
182537/182537 [==============================] - 7s 37us/sample - loss: 3.5251 - sparse_top_k_categorical_accuracy: 0.3108 - val_loss: 3.5397 - val_sparse_top_k_categorical_accuracy: 0.3117
Epoch 18/20
182537/182537 [==============================] - 7s 37us/sample - loss: 3.5251 - sparse_top_k_categorical_accuracy: 0.3108 - val_loss: 3.5397 - val_sparse_top_k_categorical_accuracy: 0.3117
Epoch 19/20
182537/182537 [==============================] - 7s 37us/sample - loss: 3.5251 - sparse_top_k_categorical_accuracy: 0.3108 - val_loss: 3.5397 - val_sparse_top_k_categorical_accuracy: 0.3117
Epoch 20/20
182537/182537 [==============================] - 7s 37us/sample - loss: 3.5251 - sparse_top_k_categorical_accuracy: 0.3108 - val_loss: 3.5397 - val_sparse_top_k_categorical_accuracy: 0.3117
7


In [24]:
resultados1 = grid_search (n_layers=[1],type_layers= ['LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
LSTM
1
64
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 8s 41us/sample - loss: 5.2082 - sparse_top_k_categorical_accuracy: 0.2697 - val_loss: 5.1510 - val_sparse_top_k_categorical_accuracy: 0.2576
Epoch 2/20
182537/182537 [==============================] - 6s 32us/sample - loss: 5.1594 - sparse_top_k_categorical_accuracy: 0.2578 - val_loss: 5.1480 - val_sparse_top_k_categorical_accuracy: 0.2576
Epoch 3/20
182537/182537 [==============================] - 6s 32us/sample - loss: 5.1584 - sparse_top_k_categorical_accuracy: 0.2578 - val_loss: 5.1489 - val_sparse_top_k_categorical_accuracy: 0.2576
Epoch 4/20
182537/182537 [==============================] - 6s 32us/sample - loss: 5.1583 - sparse_top_k_categorical_accuracy: 0.3118 - val_loss: 5.1467 - val_sparse_top_k_categorical_accuracy: 0.3781
Epoch 5/20
182537/182537 [==============================] - 6s 32us/sample - loss: 5.1176 - s

Epoch 19/20
182537/182537 [==============================] - 10s 56us/sample - loss: 7.7415 - sparse_top_k_categorical_accuracy: 0.2232 - val_loss: 7.7568 - val_sparse_top_k_categorical_accuracy: 0.2222
Epoch 20/20
182537/182537 [==============================] - 10s 56us/sample - loss: 7.7415 - sparse_top_k_categorical_accuracy: 0.2232 - val_loss: 7.7568 - val_sparse_top_k_categorical_accuracy: 0.2222
3


In [23]:
resultados2 = grid_search (n_layers=[2],type_layers= ['LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
LSTM
2
64
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 13s 68us/sample - loss: 6.8587 - sparse_top_k_categorical_accuracy: 0.2108 - val_loss: 6.8251 - val_sparse_top_k_categorical_accuracy: 0.2066
Epoch 2/20
182537/182537 [==============================] - 8s 41us/sample - loss: 6.7932 - sparse_top_k_categorical_accuracy: 0.3021 - val_loss: 6.7544 - val_sparse_top_k_categorical_accuracy: 0.2314
Epoch 3/20
182537/182537 [==============================] - 7s 41us/sample - loss: 6.7478 - sparse_top_k_categorical_accuracy: 0.2335 - val_loss: 6.7552 - val_sparse_top_k_categorical_accuracy: 0.2314
Epoch 4/20
182537/182537 [==============================] - 8s 41us/sample - loss: 6.7471 - sparse_top_k_categorical_accuracy: 0.2335 - val_loss: 6.7541 - val_sparse_top_k_categorical_accuracy: 0.2314
Epoch 5/20
182537/182537 [==============================] - 8s 41us/sample - loss: 6.7467 - 

Epoch 19/20
182537/182537 [==============================] - 14s 75us/sample - loss: 2.4849 - sparse_top_k_categorical_accuracy: 0.3068 - val_loss: 2.4849 - val_sparse_top_k_categorical_accuracy: 0.3067
Epoch 20/20
182537/182537 [==============================] - 14s 74us/sample - loss: 2.4849 - sparse_top_k_categorical_accuracy: 0.3068 - val_loss: 2.4849 - val_sparse_top_k_categorical_accuracy: 0.3067
3


In [23]:
resultados3 = grid_search (n_layers=[3],type_layers= ['LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
LSTM
3
64
3
Train on 182537 samples, validate on 60846 samples
Epoch 1/20
182537/182537 [==============================] - 13s 72us/sample - loss: 5.1514 - sparse_top_k_categorical_accuracy: 0.3056 - val_loss: 5.4893 - val_sparse_top_k_categorical_accuracy: 0.2360
Epoch 2/20
182537/182537 [==============================] - 9s 50us/sample - loss: 5.4762 - sparse_top_k_categorical_accuracy: 0.2348 - val_loss: 5.4885 - val_sparse_top_k_categorical_accuracy: 0.2360
Epoch 3/20
182537/182537 [==============================] - 9s 50us/sample - loss: 5.4770 - sparse_top_k_categorical_accuracy: 0.2348 - val_loss: 5.4904 - val_sparse_top_k_categorical_accuracy: 0.2360
Epoch 4/20
182537/182537 [==============================] - 9s 50us/sample - loss: 5.4766 - sparse_top_k_categorical_accuracy: 0.2348 - val_loss: 5.4872 - val_sparse_top_k_categorical_accuracy: 0.2360
Epoch 5/20
182537/182537 [==============================] - 9s 50us/sample - loss: 5.4766 - 

Epoch 19/20
182537/182537 [==============================] - 17s 92us/sample - loss: 3.3345 - sparse_top_k_categorical_accuracy: 0.2639 - val_loss: 3.3153 - val_sparse_top_k_categorical_accuracy: 0.2641
Epoch 20/20
182537/182537 [==============================] - 17s 92us/sample - loss: 3.3345 - sparse_top_k_categorical_accuracy: 0.2639 - val_loss: 3.3153 - val_sparse_top_k_categorical_accuracy: 0.2641
3


In [25]:
xValidation = crimes_time_validation.loc[:,'Aggravated Assault':'TIME_Y']
yValidation =  crimes_time_validation.loc[:,'DISTRICT_labels']
xValidation.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)

In [24]:
n_embeding=20

model_wo_D = tf.keras.models.Sequential()
model_wo_D.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wo_D.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D] )
model_fitted_wo_D = model_wo_D.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D = model_wo_D.evaluate(xValidation, yValidation)

Train on 182537 samples, validate on 60846 samples
Epoch 1/100
182537/182537 [==============================] - 3s 18us/sample - loss: 2.3080 - sparse_top_k_categorical_accuracy: 0.4693 - val_loss: 2.2983 - val_sparse_top_k_categorical_accuracy: 0.4747
Epoch 2/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2941 - sparse_top_k_categorical_accuracy: 0.4764 - val_loss: 2.2940 - val_sparse_top_k_categorical_accuracy: 0.4782
Epoch 3/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2904 - sparse_top_k_categorical_accuracy: 0.4783 - val_loss: 2.2942 - val_sparse_top_k_categorical_accuracy: 0.4765
Epoch 4/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2874 - sparse_top_k_categorical_accuracy: 0.4787 - val_loss: 2.2920 - val_sparse_top_k_categorical_accuracy: 0.4782
Epoch 5/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2847 - sparse_top_k_categorical_accuracy: 0.4812 - val_

182537/182537 [==============================] - 3s 15us/sample - loss: 2.1447 - sparse_top_k_categorical_accuracy: 0.5349 - val_loss: 2.4659 - val_sparse_top_k_categorical_accuracy: 0.4507
Epoch 82/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1440 - sparse_top_k_categorical_accuracy: 0.5347 - val_loss: 2.4566 - val_sparse_top_k_categorical_accuracy: 0.4517
Epoch 83/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1432 - sparse_top_k_categorical_accuracy: 0.5344 - val_loss: 2.4664 - val_sparse_top_k_categorical_accuracy: 0.4495
Epoch 84/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.1422 - sparse_top_k_categorical_accuracy: 0.5346 - val_loss: 2.4718 - val_sparse_top_k_categorical_accuracy: 0.4481
Epoch 85/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1410 - sparse_top_k_categorical_accuracy: 0.5364 - val_loss: 2.4735 - val_sparse_top_k_categorical_accuracy: 0.450

In [25]:
n_embeding=20

model_wD = tf.keras.models.Sequential()
model_wD.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wD.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD.add(tf.keras.layers.Dropout(0.25))
model_wD.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD] )
model_fitted_wD = model_wD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD = model_wD.evaluate(xValidation, yValidation)

Train on 182537 samples, validate on 60846 samples
Epoch 1/100
182537/182537 [==============================] - 3s 17us/sample - loss: 2.3106 - sparse_top_k_categorical_accuracy: 0.4664 - val_loss: 2.2985 - val_sparse_top_k_categorical_accuracy: 0.4762
Epoch 2/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2961 - sparse_top_k_categorical_accuracy: 0.4761 - val_loss: 2.2936 - val_sparse_top_k_categorical_accuracy: 0.4789
Epoch 3/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2922 - sparse_top_k_categorical_accuracy: 0.4792 - val_loss: 2.2926 - val_sparse_top_k_categorical_accuracy: 0.4768
Epoch 4/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2893 - sparse_top_k_categorical_accuracy: 0.4793 - val_loss: 2.2938 - val_sparse_top_k_categorical_accuracy: 0.4779
Epoch 5/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2872 - sparse_top_k_categorical_accuracy: 0.4800 - val_

182537/182537 [==============================] - 3s 16us/sample - loss: 2.2006 - sparse_top_k_categorical_accuracy: 0.5152 - val_loss: 2.3643 - val_sparse_top_k_categorical_accuracy: 0.4607
Epoch 82/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2007 - sparse_top_k_categorical_accuracy: 0.5139 - val_loss: 2.3625 - val_sparse_top_k_categorical_accuracy: 0.4620
Epoch 83/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.1998 - sparse_top_k_categorical_accuracy: 0.5142 - val_loss: 2.3610 - val_sparse_top_k_categorical_accuracy: 0.4620
Epoch 84/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.1994 - sparse_top_k_categorical_accuracy: 0.5155 - val_loss: 2.3614 - val_sparse_top_k_categorical_accuracy: 0.4621
Epoch 85/100
182537/182537 [==============================] - 3s 17us/sample - loss: 2.1984 - sparse_top_k_categorical_accuracy: 0.5158 - val_loss: 2.3654 - val_sparse_top_k_categorical_accuracy: 0.459

In [28]:
n_embeding=20

model_wDD = tf.keras.models.Sequential()
model_wDD.add(tf.keras.layers.Dense(512, activation='relu', input_dim=73))
model_wDD.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD] )
model_fitted_wDD = model_wDD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD = model_wDD.evaluate(xValidation, yValidation)

Train on 182537 samples, validate on 60846 samples
Epoch 1/100
182537/182537 [==============================] - 3s 18us/sample - loss: 2.3160 - sparse_top_k_categorical_accuracy: 0.4654 - val_loss: 2.2990 - val_sparse_top_k_categorical_accuracy: 0.4766
Epoch 2/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.3003 - sparse_top_k_categorical_accuracy: 0.4739 - val_loss: 2.2964 - val_sparse_top_k_categorical_accuracy: 0.4749
Epoch 3/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2971 - sparse_top_k_categorical_accuracy: 0.4752 - val_loss: 2.2939 - val_sparse_top_k_categorical_accuracy: 0.4775
Epoch 4/100
182537/182537 [==============================] - 3s 17us/sample - loss: 2.2940 - sparse_top_k_categorical_accuracy: 0.4769 - val_loss: 2.2938 - val_sparse_top_k_categorical_accuracy: 0.4773
Epoch 5/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2924 - sparse_top_k_categorical_accuracy: 0.4779 - val_

182537/182537 [==============================] - 3s 16us/sample - loss: 2.2328 - sparse_top_k_categorical_accuracy: 0.5034 - val_loss: 2.3365 - val_sparse_top_k_categorical_accuracy: 0.4661
Epoch 82/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2315 - sparse_top_k_categorical_accuracy: 0.5034 - val_loss: 2.3347 - val_sparse_top_k_categorical_accuracy: 0.4675
Epoch 83/100
182537/182537 [==============================] - 3s 17us/sample - loss: 2.2313 - sparse_top_k_categorical_accuracy: 0.5034 - val_loss: 2.3318 - val_sparse_top_k_categorical_accuracy: 0.4686
Epoch 84/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2306 - sparse_top_k_categorical_accuracy: 0.5041 - val_loss: 2.3357 - val_sparse_top_k_categorical_accuracy: 0.4666
Epoch 85/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.2301 - sparse_top_k_categorical_accuracy: 0.5044 - val_loss: 2.3325 - val_sparse_top_k_categorical_accuracy: 0.469

In [29]:
n_embeding=20

model_wo_D4 = tf.keras.models.Sequential()
model_wo_D4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wo_D4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D4] )
model_fitted_wo_D4 = model_wo_D4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D4 = model_wo_D4.evaluate(xValidation, yValidation)

Train on 182537 samples, validate on 60846 samples
Epoch 1/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.3073 - sparse_top_k_categorical_accuracy: 0.4696 - val_loss: 2.2999 - val_sparse_top_k_categorical_accuracy: 0.4742
Epoch 2/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2935 - sparse_top_k_categorical_accuracy: 0.4763 - val_loss: 2.2951 - val_sparse_top_k_categorical_accuracy: 0.4770
Epoch 3/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2896 - sparse_top_k_categorical_accuracy: 0.4789 - val_loss: 2.2929 - val_sparse_top_k_categorical_accuracy: 0.4793
Epoch 4/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2865 - sparse_top_k_categorical_accuracy: 0.4806 - val_loss: 2.2918 - val_sparse_top_k_categorical_accuracy: 0.4771
Epoch 5/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.2836 - sparse_top_k_categorical_accuracy: 0.4819 - val_

182537/182537 [==============================] - 3s 15us/sample - loss: 2.1420 - sparse_top_k_categorical_accuracy: 0.5365 - val_loss: 2.4550 - val_sparse_top_k_categorical_accuracy: 0.4482
Epoch 82/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1411 - sparse_top_k_categorical_accuracy: 0.5365 - val_loss: 2.4607 - val_sparse_top_k_categorical_accuracy: 0.4494
Epoch 83/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1403 - sparse_top_k_categorical_accuracy: 0.5359 - val_loss: 2.4601 - val_sparse_top_k_categorical_accuracy: 0.4504
Epoch 84/100
182537/182537 [==============================] - 3s 15us/sample - loss: 2.1396 - sparse_top_k_categorical_accuracy: 0.5375 - val_loss: 2.4606 - val_sparse_top_k_categorical_accuracy: 0.4462
Epoch 85/100
182537/182537 [==============================] - 3s 16us/sample - loss: 2.1381 - sparse_top_k_categorical_accuracy: 0.5371 - val_loss: 2.4661 - val_sparse_top_k_categorical_accuracy: 0.447

In [26]:
n_embeding=20

model_wD4 = tf.keras.models.Sequential()
model_wD4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wD4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD4.add(tf.keras.layers.Dropout(0.25))
model_wD4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD4] )
model_fitted_wD4 = model_wD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD4 = model_wD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.3111 - sparse_top_k_categorical_accuracy: 0.4675 - val_loss: 2.2958 - val_sparse_top_k_categorical_accuracy: 0.4764
Epoch 2/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2963 - sparse_top_k_categorical_accuracy: 0.4771 - val_loss: 2.2967 - val_sparse_top_k_categorical_accuracy: 0.4761
Epoch 3/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2921 - sparse_top_k_categorical_accuracy: 0.4777 - val_loss: 2.2918 - val_sparse_top_k_categorical_accuracy: 0.4785
Epoch 4/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2894 - sparse_top_k_categorical_accuracy: 0.4794 - val_loss: 2.2919 - val_sparse_top_k_categorical_accuracy: 0.4783
Epoch 5/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2871 - sparse_top_k_categorical_accuracy: 0.4801 - val_loss: 2.2930 - val_sparse_top_k_categorical_accuracy: 0.4775
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2295 - sparse_top_k_categorical_accuracy: 0.5042 - val_loss: 2.3271 - val_sparse_top_k_categorical_accuracy: 0.4693
Epoch 44/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2291 - sparse_top_k_categorical_accuracy: 0.5037 - val_loss: 2.3252 - val_sparse_top_k_categorical_accuracy: 0.4680
Epoch 45/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2290 - sparse_top_k_categorical_accuracy: 0.5044 - val_loss: 2.3271 - val_sparse_top_k_categorical_accuracy: 0.4681
Epoch 46/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2270 - sparse_top_k_categorical_accuracy: 0.5043 - val_loss: 2.3260 - val_sparse_top_k_categorical_accuracy: 0.4688
Epoch 47/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2273 - sparse_top_k_categorical_accuracy: 0.5046 - val_loss: 2.3281 - val_sparse_top_k_categorical_accuracy: 0.4687
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2023 - sparse_top_k_categorical_accuracy: 0.5133 - val_loss: 2.3609 - val_sparse_top_k_categorical_accuracy: 0.4630
Epoch 86/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2009 - sparse_top_k_categorical_accuracy: 0.5143 - val_loss: 2.3613 - val_sparse_top_k_categorical_accuracy: 0.4622
Epoch 87/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2005 - sparse_top_k_categorical_accuracy: 0.5143 - val_loss: 2.3673 - val_sparse_top_k_categorical_accuracy: 0.4620
Epoch 88/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2001 - sparse_top_k_categorical_accuracy: 0.5147 - val_loss: 2.3646 - val_sparse_top_k_categorical_accuracy: 0.4624
Epoch 89/100
1427/1427 [==============================] - 4s 3ms/step - loss: 2.2001 - sparse_top_k_categorical_accuracy: 0.5146 - val_loss: 2.3645 - val_sparse_top_k_categorical_accuracy: 0.4619
Epoch 90/100
1427/14

In [28]:
n_embeding=20

model_wDD4 = tf.keras.models.Sequential()
model_wDD4.add(tf.keras.layers.Dense(512, activation='relu', input_dim=73))
model_wDD4.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD4] )
model_fitted_wDD4= model_wDD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD4 = model_wDD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3159 - sparse_top_k_categorical_accuracy: 0.4659 - val_loss: 2.3006 - val_sparse_top_k_categorical_accuracy: 0.4764
Epoch 2/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.3004 - sparse_top_k_categorical_accuracy: 0.4744 - val_loss: 2.2965 - val_sparse_top_k_categorical_accuracy: 0.4778
Epoch 3/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2960 - sparse_top_k_categorical_accuracy: 0.4763 - val_loss: 2.2936 - val_sparse_top_k_categorical_accuracy: 0.4781
Epoch 4/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2934 - sparse_top_k_categorical_accuracy: 0.4766 - val_loss: 2.2937 - val_sparse_top_k_categorical_accuracy: 0.4768
Epoch 5/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2916 - sparse_top_k_categorical_accuracy: 0.4779 - val_loss: 2.2937 - val_sparse_top_k_categorical_accuracy: 0.4782
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2533 - sparse_top_k_categorical_accuracy: 0.4941 - val_loss: 2.3073 - val_sparse_top_k_categorical_accuracy: 0.4723
Epoch 44/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2525 - sparse_top_k_categorical_accuracy: 0.4951 - val_loss: 2.3084 - val_sparse_top_k_categorical_accuracy: 0.4728
Epoch 45/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2516 - sparse_top_k_categorical_accuracy: 0.4961 - val_loss: 2.3109 - val_sparse_top_k_categorical_accuracy: 0.4726
Epoch 46/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2513 - sparse_top_k_categorical_accuracy: 0.4960 - val_loss: 2.3109 - val_sparse_top_k_categorical_accuracy: 0.4735
Epoch 47/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2503 - sparse_top_k_categorical_accuracy: 0.4959 - val_loss: 2.3129 - val_sparse_top_k_categorical_accuracy: 0.4702
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2303 - sparse_top_k_categorical_accuracy: 0.5040 - val_loss: 2.3381 - val_sparse_top_k_categorical_accuracy: 0.4651
Epoch 86/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2305 - sparse_top_k_categorical_accuracy: 0.5040 - val_loss: 2.3369 - val_sparse_top_k_categorical_accuracy: 0.4641
Epoch 87/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2294 - sparse_top_k_categorical_accuracy: 0.5036 - val_loss: 2.3363 - val_sparse_top_k_categorical_accuracy: 0.4666
Epoch 88/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2292 - sparse_top_k_categorical_accuracy: 0.5044 - val_loss: 2.3364 - val_sparse_top_k_categorical_accuracy: 0.4655
Epoch 89/100
1427/1427 [==============================] - 5s 3ms/step - loss: 2.2285 - sparse_top_k_categorical_accuracy: 0.5030 - val_loss: 2.3346 - val_sparse_top_k_categorical_accuracy: 0.4654
Epoch 90/100
1427/14